In [1]:
!pip install boto3

In [2]:
import requests
import boto3
from botocore.client import Config
import json
import datetime
import locale

In [3]:
now = datetime.datetime.now(datetime.timezone.utc)
timestamp_day = now.strftime("%Y-%m-%d")
timestamp_minute = now.strftime("%Y-%m-%d-%H%M")

dataset_name = 'velib-disponibilite-en-temps-reel'

url = f"https://opendata.paris.fr/api/explore/v2.1/catalog/datasets/{dataset_name}/exports/json"
minio_url = "http://minio:9000"
access_key = "minio"
secret_key = "minio123"
bucket_name = "velib"
object_key = f"bronze/{dataset_name}/{timestamp_day}/{timestamp_minute}.json"

response = requests.get(url)
response.raise_for_status()  # Raise an error for bad responses
print(len(response.json()))
print(response.json()[0])

1485
{'stationcode': '16107', 'name': 'Benjamin Godard - Victor Hugo', 'is_installed': 'OUI', 'capacity': 35, 'numdocksavailable': 30, 'numbikesavailable': 5, 'mechanical': 1, 'ebike': 4, 'is_renting': 'OUI', 'is_returning': 'OUI', 'duedate': '2024-10-27T02:03:26+00:00', 'coordonnees_geo': {'lon': 2.275725, 'lat': 48.865983}, 'nom_arrondissement_communes': 'Paris', 'code_insee_commune': '75056'}


In [4]:
s3_client = boto3.client(
    's3',
    endpoint_url=minio_url,
    aws_access_key_id=access_key,
    aws_secret_access_key=secret_key,
    config=Config(signature_version='s3v4')
)
object_content = json.dumps(response.json())
s3_client.put_object(Bucket=bucket_name, Key=object_key, Body=object_content)
print(f"File '{object_key}' has been uploaded to bucket '{bucket_name}'.")

File 'root/bronze/velib-disponibilite-en-temps-reel/2024-10-27/2024-10-27-0351.json' has been uploaded to bucket 'velib'.
